In [1]:
from copy import deepcopy

import numpy as np
import pandas as pd
import cv2
import os
import os.path as osp

from tqdm import tqdm
from ioutils import load_pickle
from process_video import video_results_to_submission_format, process_video

%load_ext autoreload
%autoreload 2

/home/marek/miniconda3/envs/py312-molmo/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-20 11:55:59.092191: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-20 11:55:59.283573: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
annotations_path = "/home/marek/Work/COOOL_benchmark/coool/data/annotations_public.pkl"
annotations = load_pickle(annotations_path)
video_root = "/home/marek/datasets/coool-benchmark"

for video_name in annotations.keys():#Check that videos exist
    assert osp.exists(os.path.join(video_root, f"{video_name}.mp4"))


In [3]:
submission_results = {}

video_names = sorted(list(annotations.keys()))
for video_name in tqdm(video_names, total=len(video_names)):
    video_path = osp.join(video_root, f"{video_name}.mp4")
    video_results = process_video(video_path, annotations, run_tracks=True, run_captions=False)
    _submission_results = video_results_to_submission_format(video_results)
    submission_results.update(_submission_results)


df = pd.DataFrame.from_dict(submission_results, orient="index")
df.index.name = "ID"


100%|██████████| 200/200 [01:15<00:00,  2.63it/s]


In [4]:
remove_cols = ["Hazard_Track_ ", "Hazard_Name_ "]
df.drop(remove_cols, axis="columns", inplace=True, errors="ignore")
df

,Driver_State_Changed,Hazard_Track_0,Hazard_Name_0,Hazard_Track_1,Hazard_Name_1,Hazard_Track_2,Hazard_Name_2,Hazard_Track_3,Hazard_Name_3,Hazard_Track_4,...,Hazard_Track_18,Hazard_Name_18,Hazard_Track_19,Hazard_Name_19,Hazard_Track_20,Hazard_Name_20,Hazard_Track_21,Hazard_Name_21,Hazard_Track_22,Hazard_Name_22
ID,,,,,,,,,,,,,,,,,,,,,
video_0001_0,False,2,,3,,4,,5,,6,...,,,,,,,,,,
video_0001_1,False,2,,3,,4,,5,,6,...,,,,,,,,,,
video_0001_2,False,2,,3,,4,,5,,6,...,,,,,,,,,,
video_0001_3,False,2,,3,,4,,5,,6,...,,,,,,,,,,
video_0001_4,False,2,,3,,4,,5,,6,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
video_0200_365,True,1,,0,,,,,,,...,,,,,,,,,,
video_0200_366,True,1,,0,,,,,,,...,,,,,,,,,,
video_0200_367,True,1,,0,,,,,,,...,,,,,,,,,,


In [5]:
df.to_csv("../submissions/results_20122024_bboxsizes_driver_alltracks.csv", sep=",")
